In [1]:
!pip install langchain_community ollama chromadb -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.
langchain-mistralai 0.2.4 requires langchain-core<0.4.0,>=0.3.27, but you have langchain-core 0.2.43 which is incompatible.
langchain-ollama 0.2.2 requires langchain-core<0.4.0,>=0.3.27, but you have langchain-core 0.2.43 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install langchain langchainhub sentence-transformers huggingface_hub pdfplumber faiss-cpu



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install -U langchain_ollama

  Obtaining dependency information for langchain-core<0.4.0,>=0.3.27 from https://files.pythonhosted.org/packages/95/4f/fe1de63f6fc1ac7af3ba4ae12d420af1a19f7893b5fcb72856b9fc67f650/langchain_core-0.3.29-py3-none-any.whl.metadata
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.17 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.3.29 which is incompatible.
langchain-community 0.2.11 requires langchain-core<0.3.0,>=0.2.27, but you have langchain-core 0.3.29 which is incompatible.
langchain-google-genai 1.0.8 require

In [4]:
from langchain_community.llms import ollama
from langchain_community.vectorstores import chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_importer
from langchain.prompts import PromptTemplate


/Users/vaibhav/PycharmProjects/llm-visplain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/q3/mq53py8958ndz_blwtj161xm0000gn/T/ipykernel_90801/2855599551.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.llms import ollama


### Executing Embedding generation

In [20]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model="llama3")

loader = PDFPlumberLoader("/Users/vaibhav/PycharmProjects/llm-visplain/language_understanding_paper.pdf")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 200)
docs  = text_splitter.split_documents(documents)
docs


[Document(metadata={'source': '/Users/vaibhav/PycharmProjects/llm-visplain/language_understanding_paper.pdf', 'file_path': '/Users/vaibhav/PycharmProjects/llm-visplain/language_understanding_paper.pdf', 'page': 0, 'total_pages': 12, 'Author': '', 'CreationDate': 'D:20180608191434Z', 'Creator': 'LaTeX with hyperref package', 'Keywords': '', 'ModDate': 'D:20180608191434Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017) kpathsea version 6.2.3', 'Producer': 'pdfTeX-1.40.18', 'Subject': '', 'Title': '', 'Trapped': 'False'}, page_content='Improving Language Understanding\nby Generative Pre-Training\nAlecRadford KarthikNarasimhan TimSalimans IlyaSutskever\nOpenAI OpenAI OpenAI OpenAI\nalec@openai.com karthikn@openai.com tim@openai.com ilyasu@openai.com\nAbstract\nNatural language understanding comprises a wide range of diverse tasks such\nas textual entailment, question answering, semantic similarity assessment, and\ndocument classification. Although large 

### Generating embeddings

In [6]:
%pip install -qU langchain-huggingface


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")


/Users/vaibhav/PycharmProjects/llm-visplain/.venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
from langchain_community.vectorstores import Chroma

vStore = Chroma.from_documents(documents=docs, embedding=embedding)

retriever = vStore.as_retriever()

### Construct chain

In [23]:
from langchain.chains.retrieval import create_retrieval_chain

promptTemplate = PromptTemplate(
    input_variables=["context"], 
    template="""
    You are a document analyzer and very good at getting LLM architecture details from the research papers. If you are unable to find information about a particular component or parameter in the architecture, provide the architecture details as you find them. 
    This should be the flow of your answer:
    {context}
    """
)

documentChain = create_stuff_documents_chain(llm, promptTemplate)
retrievalChain = create_retrieval_chain(retriever, documentChain)

In [24]:
retrievalChain.invoke({"input": "What are the architectural components of GPT 1?"})

{'input': 'What are the architectural components of GPT 1?',
 'context': [Document(metadata={'Author': '', 'CreationDate': 'D:20180608191434Z', 'Creator': 'LaTeX with hyperref package', 'Keywords': '', 'ModDate': 'D:20180608191434Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017) kpathsea version 6.2.3', 'Producer': 'pdfTeX-1.40.18', 'Subject': '', 'Title': '', 'Trapped': 'False', 'file_path': '/Users/vaibhav/PycharmProjects/llm-visplain/language_understanding_paper.pdf', 'page': 6, 'source': '/Users/vaibhav/PycharmProjects/llm-visplain/language_understanding_paper.pdf', 'total_pages': 12}, page_content='Table4: Semanticsimilarityandclassificationresults,comparingourmodelwithcurrentstate-of-the-\nartmethods. AlltaskevaluationsinthistableweredoneusingtheGLUEbenchmark. (mc=Mathews\ncorrelation,acc=Accuracy,pc=Pearsoncorrelation)\nMethod Classification SemanticSimilarity GLUE\nCoLA SST2 MRPC STSB QQP\n(mc) (acc) (F1) (pc) (F1)\nSparsebytemLSTM[16] - 93.